In [19]:
import tensorflow as tf
import os
import os.path

class main(object):
    
    PATH_DATASET = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset'
    MFCC_DIM = 39
    BATCH_SIZE = 1
    
    def input(self, path_tfr, MFCC_DIM, BATCH_SIZE):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)
        features = tf.parse_single_example(
            serialized_example,
            features={
                "label": tf.FixedLenFeature([], tf.int64),
                "data": tf.FixedLenFeature([], tf.string),
            })
        datas, labels = tf.train.shuffle_batch(
            [tf.cast(features['data'], tf.float32), tf.cast(features['label'], tf.int32)],
            batch_size=BATCH_SIZE,capacity=10000+MFCC_DIM*BATCH_SIZE,min_after_dequeue=10000
        )
        return datas, labels

In [20]:
m = main()
m.input(os.path.join(m.PATH_DATASET, 'train-male.tfrecords'), m.MFCC_DIM, m.BATCH_SIZE)

(<tf.Tensor 'shuffle_batch_1:0' shape=(1,) dtype=float32>,
 <tf.Tensor 'shuffle_batch_1:1' shape=(1,) dtype=int32>)